In [1]:
import pyvisa
from qcodes.instrument_drivers.rohde_schwarz import (
    RohdeSchwarzZNB8,
    RohdeSchwarzZNBChannel,
)
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift
from scipy.io import savemat
from datetime import datetime, timedelta
import time

import os
Global_data_adress = r"C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA"
exp_name = 'TWPA_noise'
adress = Global_data_adress+'\\'+exp_name
if not os.path.exists(adress):
  os.mkdir(adress)
  print("Folder %s created!" % adress)
else:
  print("Folder %s already exists" % adress)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise already exists


# HEWLETT-PACKARD 33120A DC source

In [2]:
def dc_setup():
    try: print(dc_source.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        dc_source = rm.open_resource('GPIB0::3::INSTR') # GPIB connection
        print(dc_source.query('*IDN?'))

    return dc_source

def set_volt(volt):
    dc_source.write('VOLT:OFFS {}'.format(str(volt))) 

In [3]:
dc_source = dc_setup()
set_volt(2.47)

HEWLETT-PACKARD,33120A,0,8.0-5.0-1.0



# VNA init

In [4]:
def magtodb(mag): return 20*np.log10(mag)

def vna_setup_CW(cw_frequency=6e9, vna_power=-30, bandwidth=10):
    try: print(vna.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        vna = rm.open_resource('GPIB0::20::INSTR') # GPIB connection
        print(vna.query('*IDN?'))
        
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set defoult bandwidth
    vna.write('SOUR1:POW {}'.format(str(vna_power)))   # set defoult power
    vna.write('INIT1:CONT OFF')                        # single-sweep mode
    vna.write('SENS1:SWE:POIN 201')                    # set maximum possible number of points for single sweep
    vna.write(':SENS1:SWE:TYPE CW')                    # set CW mode of measurements
    return vna

def vna_set_cw(): vna.write(':SENS1:SWE:TYPE CW')    

def vna_set_lin(): vna.write(':SENS1:SWE:TYPE LIN')    

def vna_setup_sweep(cw_frequency=6e9, vna_power=-30, bandwidth=10):
    try: print(vna.query('*IDN?'))                 # check if connection already exist
    except:                                        # if no connection
        rm = pyvisa.ResourceManager()
        vna = rm.open_resource('GPIB0::20::INSTR') # GPIB connection
        print(vna.query('*IDN?'))
        
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set defoult bandwidth
    vna.write('SOUR1:POW {}'.format(str(vna_power)))   # set defoult power
    vna.write('INIT1:CONT OFF')                        # single-sweep mode
    vna.write('SENS1:SWE:POIN 201')                    # set maximum possible number of points for single sweep
    vna.write(':SENS1:SWE:TYPE CW')                    # set CW mode of measurements
    return vna

def vna_on(): vna.write('OUTP ON')
def vna_off(): vna.write('OUTP OFF')
    
def vna_meas_points(): 
    return int(vna.query('SENS1:SWE:POIN?'))  # should be 201

def vna_sweep_time(): 
    return float(vna.query('SENS1:SWE:TIME?'))  # estimated sweep time for current settings of vna

def vna_set_cw_freq(freq): 
    vna.write('SENS1:FREQ:CW {}'.format(str(freq))) # set single frequency point for CW mode in Hz
    
def vna_set_sweep_freq(freq): 
    vna.write('SENS1:FREQ:START {}'.format(str(freq[0]))) 
    vna.write('SENS1:FREQ:STOP {}'.format(str(freq[-1]))) 
    vna.write('SENS1:SWE:POIN {}'.format(str(len(freq)))) 

def vna_set_power(power): 
    vna.write('SOUR1:POW {}'.format(str(power))) # set power
    
def vna_set_band(bandwidth): 
    vna.write(':SENS1:BAND {}'.format(str(bandwidth))) # set power
    
def vna_meas_cw():
    vna_on()
    vna.write("INIT1:IMM")                              # start measurement
    time.sleep(0.2+float(vna.query('SENS1:SWE:TIME?'))) # this part is important becouse the measurement nedds to finnish before readout process
    vna.write("CALC1:FORM MLOG")                        # format of data
    data_str = vna.query("CALC1:DATA? SDAT")            # readout
    
    data = np.array(data_str.rstrip().split(",")).astype("float64")
    s = data[0::2] + 1j * data[1::2]
    i = data[0::2]
    q = data[1::2]
    return abs(s), angle(s)

In [5]:
vna = vna_setup_CW() # initializing or checking for connection

Rohde-Schwarz,ZNB20-2Port,1311601062102129,3.45



# ANAPICO init

In [7]:
def setup_anapico(power=-30, frequency = 6e9, channel = 1):
    try: 
        print(anapico.query('*IDN?'))
        print('Connection exist')
    except: 
        rm = pyvisa.ResourceManager()
#         anapico = rm.open_resource('USB0::0x03EB::0xAFFF::3C6-0B4F40003-0985::INSTR') # USB connection
        anapico = rm.open_resource('TCPIP0::169.254.5.91::inst0::INSTR') # LAN connection
        print(anapico.query('*IDN?'))
        anapico.write(r':SYST:COMM:LAN:RTMO {}'.format(str(1)))
    if power<=0:
#         print('Flux power set:', power)
        anapico.write(r'SOUR{}:POW {}'.format(str(channel), str(power)))
        anapico.write(r'SOUR{}:FREQ {}'.format(str(channel), str(frequency)))
    else: 
        print('Flux power is too high:', power)
    return anapico

def anapico_on(channel=1):
    anapico.write(r'OUTP{} ON'.format(str(channel)))

def anapico_off(channel=1):
    anapico.write(r'OUTP{} OFF'.format(str(channel)))

def anapico_freq(freq, channel=1): 
    anapico.write(r'SOUR{}:FREQ {}'.format(str(channel), str(freq)))

def anapico_power(power, channel=1): 
    anapico.write(r'SOUR{}:POW {}'.format(str(channel), str(power)))

In [16]:
anapico = setup_anapico()

AnaPico AG,APMS20G,3C6-0B4F40003-0985,0.4.204



In [21]:
anapico.close()

# ANRITSU MS2830A signal analizer init

In [9]:
def setup_anri():
    try: print(anri.query('*IDN?'))
    except: 
        rm = pyvisa.ResourceManager()
        anri = rm.open_resource('GPIB0::7::INSTR') # GPIB connection
        print(anri.query('*IDN?'))
    return anri


def anri_set_cent_freq(freq): 
    anri.write('FREQ:CENT {}'.format(str(freq)))
    
def anri_set_span(span): 
    anri.write('FREQ:SPAN {}'.format(str(span)))
    
def anri_set_band_kHz(band): 
    anri.write('BAND {}KHZ'.format(str(band)))

def anri_set_band_Hz(band): 
    anri.write('BAND {}HZ'.format(str(band)))
    
def anri_set_nop(nop):
    # max 10001
    anri.write('SWEep:POINts {}'.format(str(nop)))
    
    
def anri_get_nop():
    return int(anri.query('SWEep:POINts?'))

def anri_get_sweep_time():
    return float(anri.query('SWEep:TIME?'))

def sweep_mode_cont():
    anri.write('INIT:MODE:CONT')

def sweep_mode_sing():
    anri.write('INIT:MODE:SING')

def anri_get_data():
    anri.write('INIT:IMM')
    time.sleep(20+float(anri.query('SWEep:TIME?')))
    raw_data = anri.query('TRAC? TRAC1')
    data = np.array(raw_data.split(','), dtype = float)
    return data

In [10]:
anri = setup_anri()

ANRITSU,MS2830A,6200847105,4.09.01



Create log

In [159]:
comment = 'Intermodulation product with offset in flux frequency'


file_name = adress +'\\'+'log_'+exp_name+'.txt'
f = open(file_name, 'a')  # append  to existing file
f.write(str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))+':   ')
f.write(comment+'\n')
f.close()

In [11]:
now = datetime.now()
path = adress +'\\'+ 'IntProd_DC_sweep_fp_m5_fm_m28'
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\IntProd_DC_sweep_fp_m5_fm_m28 created!


In [12]:
now1 = datetime.now()
print('Start: {}'.format(now1.strftime(r'%H-%M-%S')))

flux_freq = 11.252e9
flux_power = -5
anapico_power(flux_power, 1)
anapico_freq(flux_freq, 1)
anapico_on(1)

mode1 = flux_freq/2+3e6
anapico_freq(mode1, 2)

mode2 = flux_freq/2+4e6
anapico_freq(mode2, 3)

band = 10
cent_freq = flux_freq/2
span = 45e6
nop = 10001
min_freq = cent_freq-span/2
max_freq = cent_freq+span/2
freq = np.linspace(min_freq, max_freq, nop)

# anri_set_band_kHz(band)
anri_set_nop(nop)
anri_set_band_Hz(band)
anri_set_cent_freq(cent_freq)
anri_set_span(span)

powers = np.array([-28], dtype=float)
mode1_power = powers[0]
anapico_power(mode1_power, 2)
anapico_on(2)

mode2_power = powers[0]
anapico_power(mode2_power, 3)
anapico_on(3)

volts = np.linspace(1.5,3.5,90)
print('est runtime {}'.format(str(timedelta(seconds=8*len(volts)))))

for v in volts:
    
    set_volt(v)

    now_curr = datetime.now()
    print('New parameters {} set at {} '.format(v, now_curr.strftime(r'%H-%M-%S')))
    data = anri_get_data()
    print('max:', max(data))
    
    dict_data=dict()
    dict_data['flux_power'] = flux_power
    dict_data['flux_freq'] = flux_freq
    dict_data['mode1'] = mode1
    dict_data['mode1_power'] = mode1_power
    dict_data['mode2'] = mode2
    dict_data['mode2_power'] = mode2_power

    dict_data['band_Hz'] = band
    dict_data['freq_sweep'] = freq
    dict_data['nop'] = nop 
    
    dict_data['volt'] = v
    dict_data['data'] = data

    experiment_name = 'InterProd_powers {}-{}-{}__Dc_{}'.format(flux_power, mode1_power, mode2_power, v)
    file_name = path+'\\'+experiment_name+now.strftime(r'%H-%M-%S')+'.mat'
    savemat(file_name, dict_data)
    
                    
now2 = datetime.now()
print('Stop: {}'.format(now2.strftime(r'%H-%M-%S'))) 
print('Run time:', now2-now1)
anapico.close()

Start: 17-00-40
est runtime 0:12:00
New parameters 1.5 set at 17-00-40 
max: -33.298
New parameters 1.5224719101123596 set at 17-09-29 
max: -33.233
New parameters 1.5449438202247192 set at 17-18-17 
max: -33.135
New parameters 1.5674157303370786 set at 17-27-06 
max: -33.082
New parameters 1.5898876404494382 set at 17-35-54 
max: -32.961
New parameters 1.6123595505617978 set at 17-44-43 
max: -32.866
New parameters 1.6348314606741572 set at 17-53-31 
max: -32.807
New parameters 1.6573033707865168 set at 18-02-19 
max: -32.695
New parameters 1.6797752808988764 set at 18-11-08 
max: -32.617
New parameters 1.702247191011236 set at 18-19-56 
max: -32.556
New parameters 1.7247191011235956 set at 18-28-45 
max: -32.533
New parameters 1.747191011235955 set at 18-37-33 
max: -32.614
New parameters 1.7696629213483146 set at 18-46-22 
max: -33.987
New parameters 1.7921348314606742 set at 18-55-10 
max: -35.1
New parameters 1.8146067415730336 set at 19-03-58 
max: -35.906
New parameters 1.837078

In [17]:
now = datetime.now()
path = adress +'\\'+ 'IntProd_DC_sweep_fp_m5_fm_m24'
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\IntProd_DC_sweep_fp_m5_fm_m24 already exists


In [18]:
now1 = datetime.now()
print('Start: {}'.format(now1.strftime(r'%H-%M-%S')))

flux_freq = 11.252e9
flux_power = -5
anapico_power(flux_power, 1)
anapico_freq(flux_freq, 1)
anapico_on(1)

mode1 = flux_freq/2+3e6
anapico_freq(mode1, 2)

mode2 = flux_freq/2+4e6
anapico_freq(mode2, 3)

band = 10
cent_freq = flux_freq/2
span = 45e6
nop = 10001
min_freq = cent_freq-span/2
max_freq = cent_freq+span/2
freq = np.linspace(min_freq, max_freq, nop)

# anri_set_band_kHz(band)
anri_set_nop(nop)
anri_set_band_Hz(band)
anri_set_cent_freq(cent_freq)
anri_set_span(span)

powers = np.array([-24], dtype=float)
mode1_power = powers[0]
anapico_power(mode1_power, 2)
anapico_on(2)

mode2_power = powers[0]
anapico_power(mode2_power, 3)
anapico_on(3)

volts = np.linspace(1.5,3.5,90)
print('est runtime {}'.format(str(timedelta(seconds=8*len(volts)))))

for v in volts:
    
    set_volt(v)

    now_curr = datetime.now()
    print('New parameters {} set at {} '.format(v, now_curr.strftime(r'%H-%M-%S')))
    data = anri_get_data()
    print('max:', max(data))
    
    dict_data=dict()
    dict_data['flux_power'] = flux_power
    dict_data['flux_freq'] = flux_freq
    dict_data['mode1'] = mode1
    dict_data['mode1_power'] = mode1_power
    dict_data['mode2'] = mode2
    dict_data['mode2_power'] = mode2_power

    dict_data['band_Hz'] = band
    dict_data['freq_sweep'] = freq
    dict_data['nop'] = nop 
    
    dict_data['volt'] = v
    dict_data['data'] = data

    experiment_name = 'InterProd_powers {}-{}-{}__Dc_{}'.format(flux_power, mode1_power, mode2_power, v)
    file_name = path+'\\'+experiment_name+now.strftime(r'%H-%M-%S')+'.mat'
    savemat(file_name, dict_data)
    
                    
now2 = datetime.now()
print('Stop: {}'.format(now2.strftime(r'%H-%M-%S'))) 
print('Run time:', now2-now1)

Start: 14-01-32
est runtime 0:12:00
New parameters 1.5 set at 14-01-32 
max: -25.999
New parameters 1.5224719101123596 set at 14-10-21 
max: -25.934
New parameters 1.5449438202247192 set at 14-19-09 
max: -25.867
New parameters 1.5674157303370786 set at 14-27-58 
max: -25.806
New parameters 1.5898876404494382 set at 14-36-46 
max: -25.739
New parameters 1.6123595505617978 set at 14-45-36 
max: -25.657
New parameters 1.6348314606741572 set at 14-54-24 
max: -25.583
New parameters 1.6573033707865168 set at 15-03-13 
max: -25.509
New parameters 1.6797752808988764 set at 15-12-01 
max: -25.431
New parameters 1.702247191011236 set at 15-20-51 
max: -25.38
New parameters 1.7247191011235956 set at 15-29-39 
max: -25.35
New parameters 1.747191011235955 set at 15-38-28 
max: -25.404
New parameters 1.7696629213483146 set at 15-47-16 
max: -26.892
New parameters 1.7921348314606742 set at 15-56-06 
max: -27.908
New parameters 1.8146067415730336 set at 16-04-54 
max: -28.751
New parameters 1.837078

In [19]:
now = datetime.now()
path = adress +'\\'+ 'IntProd_DC_sweep_fp_m5_fm_m22'
if not os.path.exists(path):
  os.mkdir(path)
  print("Folder %s created!" % path)
else:
  print("Folder %s already exists" % path)

Folder C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\IntProd_DC_sweep_fp_m5_fm_m22 created!


In [20]:
now1 = datetime.now()
print('Start: {}'.format(now1.strftime(r'%H-%M-%S')))

flux_freq = 11.252e9
flux_power = -5
anapico_power(flux_power, 1)
anapico_freq(flux_freq, 1)
anapico_on(1)

mode1 = flux_freq/2+3e6
anapico_freq(mode1, 2)

mode2 = flux_freq/2+4e6
anapico_freq(mode2, 3)

band = 10
cent_freq = flux_freq/2
span = 45e6
nop = 10001
min_freq = cent_freq-span/2
max_freq = cent_freq+span/2
freq = np.linspace(min_freq, max_freq, nop)

# anri_set_band_kHz(band)
anri_set_nop(nop)
anri_set_band_Hz(band)
anri_set_cent_freq(cent_freq)
anri_set_span(span)

powers = np.array([-22], dtype=float)
mode1_power = powers[0]
anapico_power(mode1_power, 2)
anapico_on(2)

mode2_power = powers[0]
anapico_power(mode2_power, 3)
anapico_on(3)

volts = np.linspace(1.5,3.5,90)
print('est runtime {}'.format(str(timedelta(seconds=8*len(volts)))))

for v in volts:
    
    set_volt(v)

    now_curr = datetime.now()
    print('New parameters {} set at {} '.format(v, now_curr.strftime(r'%H-%M-%S')))
    data = anri_get_data()
    print('max:', max(data))
    
    dict_data=dict()
    dict_data['flux_power'] = flux_power
    dict_data['flux_freq'] = flux_freq
    dict_data['mode1'] = mode1
    dict_data['mode1_power'] = mode1_power
    dict_data['mode2'] = mode2
    dict_data['mode2_power'] = mode2_power

    dict_data['band_Hz'] = band
    dict_data['freq_sweep'] = freq
    dict_data['nop'] = nop 
    
    dict_data['volt'] = v
    dict_data['data'] = data

    experiment_name = 'InterProd_powers {}-{}-{}__Dc_{}'.format(flux_power, mode1_power, mode2_power, v)
    file_name = path+'\\'+experiment_name+now.strftime(r'%H-%M-%S')+'.mat'
    savemat(file_name, dict_data)
    
                    
now2 = datetime.now()
print('Stop: {}'.format(now2.strftime(r'%H-%M-%S'))) 
print('Run time:', now2-now1)

Start: 03-14-36
est runtime 0:12:00
New parameters 1.5 set at 03-14-36 
max: -23.68
New parameters 1.5224719101123596 set at 03-23-25 
max: -23.612
New parameters 1.5449438202247192 set at 03-32-14 
max: -23.532
New parameters 1.5674157303370786 set at 03-41-02 
max: -23.458
New parameters 1.5898876404494382 set at 03-49-51 
max: -23.389
New parameters 1.6123595505617978 set at 03-58-39 
max: -23.323
New parameters 1.6348314606741572 set at 04-07-27 
max: -23.254
New parameters 1.6573033707865168 set at 04-16-16 
max: -23.139
New parameters 1.6797752808988764 set at 04-25-05 
max: -23.072
New parameters 1.702247191011236 set at 04-33-54 
max: -23.049
New parameters 1.7247191011235956 set at 04-42-42 
max: -23.021
New parameters 1.747191011235955 set at 04-51-31 
max: -23.094
New parameters 1.7696629213483146 set at 05-00-21 
max: -24.545
New parameters 1.7921348314606742 set at 05-09-10 
max: -25.618
New parameters 1.8146067415730336 set at 05-17-58 
max: -26.475
New parameters 1.83707